# Evidence relation classification using Siamese BERT network

TLDR:

- Using `sentence-transformers/msmarco-bert-base-dot-v5`
- Based on [nils_2019](https://arxiv.org/abs/1908.10084)

References:

- Hugging Face [semantic search][semantic_search_guide]
- SBERT [pre-trained models][sbert_pretrained_models]
- Installation: [`sentence-transformers`][sentence_transformers]
- Code reference [`semantic_search.py`][semantic_search_py]
- Model info: [`msmarco-bert-base-dot-v5`][msmarco-bert-base-dot-v5]



[semantic_search_guide]: https://www.sbert.net/examples/applications/semantic-search/README.html
[sbert_pretrained_models]: https://www.sbert.net/docs/pretrained_models.html#msmarco-passage-models
[sentence_transformers]: https://www.sbert.net/docs/installation.html
[semantic_search_py]: https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/semantic-search/semantic_search.py
[msmarco-bert-base-dot-v5]: https://huggingface.co/sentence-transformers/msmarco-bert-base-dot-v5

## Setup

### Working directory

In [2]:
# Change the working directory to project root
import pathlib
import os
ROOT_DIR = pathlib.Path.cwd().parent
os.chdir(ROOT_DIR)

### Dependencies

In [3]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from src.torch_utils import get_torch_device
from src.data import load_as_dataframe, slice_by_claim, \
    get_claim_texts, get_evidence_texts, get_paired_texts

/opt/homebrew/Caskroom/miniconda/base/envs/comp90042_project/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Get the best torch device
torch_device = get_torch_device()

Torch device is 'mps'


In [5]:
# Directories for saving/loading
EMB_PATH = ROOT_DIR.joinpath("result", "embeddings", "*")
print(EMB_PATH)

MODEL_PATH = ROOT_DIR.joinpath("result", "models", "*")
print(MODEL_PATH)

/Users/johnsonzhou/git/comp90042-project/result/embeddings/*
/Users/johnsonzhou/git/comp90042-project/result/models/*


## Load Data

### Load from json

In [6]:
data_names = ["train-claims", "dev-claims", "evidence"]
train_claims, dev_claims, all_evidences \
    = load_as_dataframe(data_names, full_evidence=True)

Loaded train-claims
Loaded dev-claims
Loaded evidence


### Sample inspection

In [7]:
sample_claim = slice_by_claim(train_claims, None, 2, ["SUPPORTS", "DISPUTED"])
sample_claim

evidence_text
claim      claim_text                                         claim_label evidences                                                          
claim-1937 Not only is there no scientific evidence that C... DISPUTED    evidence-442946   At very high concentrations (100 times atmosph...
                                                                          evidence-1194317  Plants can grow as much as 50 percent faster i...
                                                                          evidence-12171    Higher carbon dioxide concentrations will favo...
claim-2510 In 1946, PDO switched to a cool phase.             SUPPORTS    evidence-530063   There is evidence of reversals in the prevaili...
                                                                          evidence-984887   1945/1946: The PDO changed to a "cool" phase, ...

In [8]:
sample_claim_texts = get_claim_texts(sample_claim)
sample_claim_texts

['Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.',
 'In 1946, PDO switched to a cool phase.']

In [9]:
sample_evidence_texts = get_evidence_texts(sample_claim)
sample_evidence_texts

['At very high concentrations (100 times atmospheric concentration, or greater), carbon dioxide can be toxic to animal life, so raising the concentration to 10,000 ppm (1%) or higher for several hours will eliminate pests such as whiteflies and spider mites in a greenhouse.',
 'Plants can grow as much as 50 percent faster in concentrations of 1,000 ppm CO 2 when compared with ambient conditions, though this assumes no change in climate and no limitation on other nutrients.',
 'Higher carbon dioxide concentrations will favourably affect plant growth and demand for water.',
 'There is evidence of reversals in the prevailing polarity (meaning changes in cool surface waters versus warm surface waters within the region) of the oscillation occurring around 1925, 1947, and 1977; the last two reversals corresponded with dramatic shifts in salmon production regimes in the North Pacific Ocean.',
 '1945/1946: The PDO changed to a "cool" phase, the pattern of this regime shift is similar to the 19

## Select HF embedding model

For our task, we generally have a **short** query of claims and **long** evidence passages. Therefore, we are conducting **asymmetric semantic search**. Based on the guide, MSMARCO models are most suited for this task.

In [10]:
embedder = SentenceTransformer(
    "sentence-transformers/msmarco-bert-base-dot-v5",
    device=torch_device
)
embedder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)